In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("NWM_data_updated_land1.xlsx", engine='openpyxl')
a=xl.sheet_names

                                                                                                                                
sheet_no = list(range(2, 32))

In [2]:
#loading the data set
dataframe_collection = {}
i=0
for i in range(33):
    if i==0 :
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])


dataframe_collection[0]=dataframe_collection[0].dropna().reset_index(drop=True)

In [3]:
dataframe_collection[0]

,STAID_1,Dist2Sea(degree),LandUse,SoilTexture-1layer,SoilTexture-2layer,SoilTexture-3layer,SoilTexture-4layer,SoilTexture-5layer,SoilTexture-6layer,SoilTexture-7layer,SoilTexture-8layer,SoilTexture-9layer,SoilTexture-10layer,TopographyIndex,DrainageArea(sqml)
0,02217274,3.974170,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.00
1,02218565,3.899150,1.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,7.584810,176.00
2,02334480,4.011274,1.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,6.998782,5.06
3,02334578,3.974920,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.472837,6.90
4,02334620,3.948711,1.0,3.0,3.0,3.0,12.0,12.0,12.0,12.0,12.0,3.0,15.0,7.122812,47.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,02458600,2.941943,1.0,4.0,4.0,4.0,4.0,4.0,9.0,8.0,8.0,8.0,15.0,8.280088,52.50
76,02423397,2.963616,1.0,4.0,4.0,4.0,4.0,4.0,8.0,2.0,16.0,15.0,15.0,7.952460,24.40
77,02423586,2.848893,1.0,4.0,4.0,4.0,4.0,6.0,6.0,9.0,9.0,15.0,15.0,7.293037,261.00
78,02423630,2.719344,1.0,4.0,4.0,4.0,4.0,6.0,6.0,9.0,9.0,15.0,15.0,7.444744,1027.00


In [4]:

x_mean = pd.read_csv("x_mean.csv", sep='\t')
x_std =  pd.read_csv("x_std.csv", sep='\t')
y_mean =  pd.read_csv("y_mean.csv", sep='\t')
y_std =  pd.read_csv("y_std.csv", sep='\t')

In [5]:
from keras.models import load_model
import statistics
import math
pred = [None] * 30
std = [None] * 30
std_err = [None] *30
obsflow = dataframe_collection[2]
for l in range(0,30) :
     
    model = load_model('my_model_%d_day.h5'%(l))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    modelflow= dataframe_collection[l+3]
    temp =  [0] * 80
    for index, row in dataframe_collection[0].iterrows():
        
        
        column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
        df = pd.DataFrame(columns = column_names)  
        k=0
        d=0
      
        sta_id = dataframe_collection[0].iloc[index,0]
        dis2sea=dataframe_collection[0].iloc[index,1]
        land_use=dataframe_collection[0].iloc[index,2]
        soil_texture1=dataframe_collection[0].iloc[index,3]
        soil_texture2=dataframe_collection[0].iloc[index,4]
        soil_texture3=dataframe_collection[0].iloc[index,5]
        soil_texture4=dataframe_collection[0].iloc[index,6]
        soil_texture5=dataframe_collection[0].iloc[index,7]
        soil_texture6=dataframe_collection[0].iloc[index,8]
        soil_texture7=dataframe_collection[0].iloc[index,9]
        soil_texture8=dataframe_collection[0].iloc[index,10]
        soil_texture9=dataframe_collection[0].iloc[index,11]
        soil_texture10=dataframe_collection[0].iloc[index,12]
        topoindex = dataframe_collection[0].iloc[index,13]
        drainagearea =  dataframe_collection[0].iloc[index,14]
        obs = obsflow[sta_id]/drainagearea
        mod = modelflow[sta_id]/drainagearea

        for i in range(481,979):
            df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
            k=k+1
        df =df.dropna()
            
        if df.empty:
            temp[index] = 0
            d=d+1
            print(index)
        else :
            x, y = df.iloc[ :, 0:15], df.iloc[:,15]
            x-= x_mean.iloc[l,1:16]
            x /= x_std.iloc[l,1:16]
            
            z=model.predict(x)
            
            x=x*x_std.iloc[l,1:16]
            x=x +x_mean.iloc[l,1:16]
            
            z-= y_mean.iloc[l,1]
            z /= y_std.iloc[l,1]
            
            z=z.ravel()
            z=abs(z)
            a=x.iloc[:,14]
            b=a-(2*z)
            c=a+(z*2)
            res=y.between(b,c)
            count = np.count_nonzero(res)
            temp[index] = count/len(y)   
       
    std[l]= statistics.stdev(temp)    
    pred[l] = sum(temp)/(80-d) 
    std_err[l] = (std[l]*2)/math.sqrt(80-d)
    del model


2022-01-05 23:20:41.660467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/dav/opt/cuda/11.4.0/extras/CUPTI/lib64:/glade/u/apps/dav/opt/cuda/11.4.0/lib64:/glade/u/apps/dav/opt/openmpi/4.1.1/intel/19.1.1/lib:/glade/u/apps/dav/opt/ucx/1.11.0/lib:/glade/u/apps/opt/intel/2020u1/compilers_and_libraries/linux/lib/intel64:/glade/u/apps/dav/opt/usr/lib64
2022-01-05 23:20:41.660516: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-05 23:20:41.661489: I tensorflow/core/platform/cpu_feature_guard.cc:15

56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56
56


In [6]:
d_x= pd.DataFrame(pred)
d_y=pd.DataFrame(std)
d_z=pd.DataFrame(std_err)
d_xy=pd.concat([d_x, d_y, d_z], axis=1)
d_xy.columns = ['Pred', 'Std', 'Std_err']
d_xy
d_xy.to_csv("prediction_std_stderr_probability_lnd1.csv")

In [8]:
std

[0.43129116236079695,
 0.4306611883314265,
 0.44056221320650135,
 0.4346104318629076,
 0.4395161116347075,
 0.43363295397296014,
 0.4328945768219514,
 0.4421505017424806,
 0.43551030591749473,
 0.43605783356735434,
 0.43557165594155345,
 0.4342998151681211,
 0.4353403190593076,
 0.4310754951727763,
 0.43535219618860704,
 0.43377654462344317,
 0.4371143114765151,
 0.4331519166570695,
 0.4344135748824627,
 0.4352470245954111,
 0.43170513437227487,
 0.43602212701711984,
 0.4355817024790675,
 0.4364158007669912,
 0.434525658718212,
 0.43590393905224817,
 0.4324500814212441,
 0.4339138280121154,
 0.43309329232948646,
 0.43639565548144027]

In [63]:
temp

[1.0,
 1.0,
 1.0,
 0.1487964989059081,
 1.0,
 1.0,
 0.9803063457330415,
 1.0,
 0.08533916849015317,
 0.7702407002188184,
 1.0,
 0.9824561403508771,
 0.9934354485776805,
 0.912472647702407,
 0.9956236323851203,
 0.824945295404814,
 0.9890590809628009,
 1.0,
 0.9978118161925602,
 0.9912472647702407,
 0.05689277899343545,
 0.9978118161925602,
 0.10722100656455143,
 0.9824945295404814,
 0.9080962800875274,
 0.9803063457330415,
 0.9939024390243902,
 1.0,
 0.25164113785557984,
 0.9715536105032823,
 0.1772428884026258,
 0.975929978118162,
 0.986870897155361,
 0.48358862144420134,
 1.0,
 0.9714912280701754,
 0.8796498905908097,
 0.9934354485776805,
 0.03325942350332594,
 0.9606126914660832,
 0.962800875273523,
 0.4780701754385965,
 0.0525164113785558,
 0.9912472647702407,
 0.5382932166301969,
 0.9978118161925602,
 0.12472647702407003,
 1.0,
 0.24507658643326038,
 1.0,
 0.9934065934065934,
 0.990990990990991,
 0.899343544857768,
 0,
 1.0,
 0.9934354485776805,
 1.0,
 0.9212253829321663,
 0.98468

In [40]:
x, y = df.iloc[ :, 0:15], df.iloc[:,15]
x

,dis2sea,landUse,soilTexture1,soilTexture2,soilTexture3,soilTexture4,soilTexture5,soilTexture6,soilTexture7,soilTexture8,soilTexture9,soilTexture10,topoIndex,drainageArea,ModelFlow
0,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.002317
1,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.002179
2,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.002127
3,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.001950
4,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.001835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.001118
494,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.001170
495,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.001376
496,3.97417,1.0,3.0,3.0,3.0,7.0,12.0,12.0,12.0,12.0,3.0,15.0,7.578771,109.0,0.001376


In [55]:
pred

[0      1.066681
 1      1.066658
 2      1.066648
 3      1.066637
 4      1.066663
          ...   
 493    1.066585
 494    1.066539
 495    1.066513
 496    1.066542
 497    1.066592
 Name: ModelFlow, Length: 498, dtype: float64,
 0      1.003618
 1      1.003597
 2      1.003577
 3      1.003556
 4      1.003551
          ...   
 493    1.003363
 494    1.003373
 495    1.003402
 496    1.003388
 497    1.003379
 Name: ModelFlow, Length: 457, dtype: float64,
 0      0.609107
 1      0.609095
 2      0.609083
 3      0.609071
 4      0.609060
          ...   
 493    0.608952
 494    0.608952
 495    0.608975
 496    0.608968
 497    0.608963
 Name: ModelFlow, Length: 457, dtype: float64,
 0      0.284283
 1      0.284277
 2      0.284275
 3      0.284268
 4      0.284262
          ...   
 493    0.284210
 494    0.284210
 495    0.284210
 496    0.284218
 497    0.284215
 Name: ModelFlow, Length: 457, dtype: float64,
 0      0.277445
 1      0.277441
 2      0.277444
 3      0.277